In [1]:
%pip install google-generativeai langchain langchain_google_genai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from dotenv import load_dotenv

# Load API key from .env file
load_dotenv()

True

In [3]:
import os
import json
import re
import google.generativeai as genai

# Step 1: Setup Gemini API
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

In [30]:
problem_statement = """
Metro station wants to establish a TicketDistributor machine that issues tickets for
passengers travelling on metro rails. Travellers have options of selecting a ticket for a single
trip, round trips or multiple trips. They can also issue a metro pass for regular passengers or
a time card for a day, a week or a month according to their requirements. The discounts on
tickets will be provided to frequent travelling passengers. The machine is also supposed to
read the metro pass and time cards issued by the metro counters or machine. The ticket rates
differ based on whether the traveller is a child or an adult. The machine is also required to
recognize original as well as fake currency notes. The typical transaction consists of a user
using the display interface to select the type and quantity of tickets and then choosing a
payment method of either cash, credit/debit card or smartcard. The tickets are printed and
dispensed to the user. Also, the messaging facilities after every transaction are required on
the registered number. The system can also be operated comfortably by a touch-screen. A
large number of heavy components are to be used. We do not want our system to slow down,
and also the usability of the machine.
The TicketDistributor must be able to handle several exceptions, such as aborting the
transaction for incomplete transactions, the insufficient amount given by the travellers to the
machine, money return in case of an aborted transaction, change return after a successful
transaction, showing insufficient balance in the card, updated information printed on the
tickets e.g. departure time, date, time, price, valid from, valid till, validity duration, ticket
issued from and destination station. In case of exceptions, an error message is to be displayed.
We do not want user feedback after every development stage but after every two stages to
save time. The machine is required to work in a heavy load environment such that in the
morning and evening time on weekdays, and weekends performance and efficiency would
not be affected."""

### Step 1 - Identify Actions

In [31]:
def identify_actions(problem_statement):
    """
    Identifies potential actions from a given problem statement using Gemini API with CoT prompting.
    """
    prompt = f"""
    You are an expert software analyst. Your task is to identify **key actions** from the given problem statement by carefully reasoning step by step. 

    Your answer must reflect deliberate thinking and analysis.

    ---

    ### 🧠 Step-by-Step Chain-of-Thought Reasoning:

    1. **Extract Action Candidates:**
    - Carefully read the problem statement and highlight all phrases or sentences involving **verbs** that indicate actions performed by users, systems, or external actors.

    2. **Analyze Each Candidate:**
    - For each verb or verb phrase, ask:
        - Is this a meaningful system interaction or behavior?
        - Does it result in a **state change** or **data processing**?
        - Is this relevant to the core flow of the system?

    3. **Refine and Normalize Actions:**
    - Convert each valid action to **camelCase**.
    - Discard vague or generic verbs like “handle,” “process,” unless used in a well-defined context.
    - Merge similar or redundant actions into a single representative form.

    4. **Validate Coverage and Completeness:**
    - Re-read the problem statement to ensure no important action is missing.
    - Double-check if the identified actions represent **all user and system behaviors** that are essential to the system flow.

    ---

    ### 🧾 Intermediate Reasoning Output (Example Format):
    Write the reasoning for **each action** like this before outputting JSON:

    - "registerUser": derived from "The user fills out a form and submits it to register an account"
    - "verifyEmail": derived from "The system sends a verification link to the user's email"

    (Do this for each identified action to explain how you derived it.)

    ---

    ### ✅ Final Output Format (Strict JSON Only):
    After your reasoning, output valid JSON in this format (on a new line):

    {{
        "actions": ["registerUser", "verifyEmail", "loginUser"]
    }}

    ---

    **Problem Statement:**
    {problem_statement}

    Now follow the above steps to think step by step and extract the most accurate and complete list of actions.
    """


    try:
        # Generate response using Gemini model
        model = genai.GenerativeModel("gemini-1.5-flash")
        response = model.generate_content(prompt, generation_config={"temperature": 0.0, "top_p": 1, "top_k": 1})
        
        # Debugging: Print raw response
        response_text = response.text.strip()
        print("Raw Response from Gemini:\n", response_text)

        # Clean up response (remove markdown formatting)
        cleaned_text = re.sub(r"```json|```", "", response_text).strip()

        # Extract the JSON part using regex
        json_match = re.search(r"\{\s*\"actions\".*\}", cleaned_text, re.DOTALL)
        if json_match:
            json_str = json_match.group(0)

            try:
                output = json.loads(json_str)
                return output.get("actions", [])
            except json.JSONDecodeError as e:
                print("JSON parsing error:", str(e))
                return {}
        else:
            print("No JSON found in response.")
            return {}

        # Parse JSON safely
        output = json.loads(cleaned_text)
        return output.get("actions", [])

    except Exception as e:
        print("Error:", str(e))
        return {}

# Identify actions using Gemini
identified_actions = identify_actions(problem_statement)

# Display Result
print("\nIdentified Actions:", identified_actions)


Raw Response from Gemini:
 - "selectTicketType": derived from "Travellers have options of selecting a ticket for a single trip, round trips or multiple trips. They can also issue a metro pass for regular passengers or a time card for a day, a week or a month according to their requirements."  This encompasses the user's choice of ticket type.

- "selectTicketQuantity": derived from "The typical transaction consists of a user using the display interface to select the type and quantity of tickets..." This represents the user specifying how many tickets they need.

- "selectPaymentMethod": derived from "...and then choosing a payment method of either cash, credit/debit card or smartcard." This captures the user's payment selection.

- "processCashPayment": derived from "The machine is also required to recognize original as well as fake currency notes." and "change return after a successful transaction". This covers cash handling, including authentication and change dispensing.

- "process

### Step 2 - Define Activity Nodes

In [14]:
def define_activity_nodes(actions):
    """
    Identifies activity nodes from given actions using Gemini API with CoT prompting.
    """
    prompt = f"""
    You are an expert software analyst. Your task is to define **activity diagram nodes** from a list of identified actions. 
    Use structured reasoning and classification to ensure correctness and completeness.

    ---

    ### 🧠 Step-by-Step Chain-of-Thought Reasoning:

    1. **Analyze Actions Sequentially:**
    - Think about the natural flow of the system.
    - Identify which action marks the **starting point** of the process.
    - Determine which actions represent decisions, parallel flows, or endpoints.

    2. **Map Each Action to a Node Type:**
    - Action that starts the flow → **Initial Node**.
    - Regular process or task → **Action Node**.
    - Choice or condition → **Decision Node**.
    - Merging alternate paths → **Merge Node**.
    - Parallel task initiation → **Fork Node**.
    - Parallel task synchronization → **Join Node**.
    - End of process → **Final Node**.

    3. **Assign Meaningful Names:**
    - Use meaningful and concise camelCase names for node `name` values.
    - Ensure they clearly describe what happens at each node.

    4. **Ensure Diagram Completeness:**
    - Include at least: 1 Initial Node, 1 Final Node.
    - All critical actions should be represented as nodes.
    - Include control nodes where logical branching or parallelism occurs.

    ---

    ### ✨ Reasoning Example:
    - "selectTicketType" → Action Node: basic interaction.
    - "choosePaymentMethod" → Decision Node: user selects between methods.
    - "processCashPayment" and "processCardPayment" → Action Nodes, may follow a Fork or Decision.
    - "printTicket" → likely comes near the end.
    - "displayErrorMessage" → may be downstream of a Decision Node for failure cases.
    (Do this for each action before JSON output.)

    ---

    ### ✅ Output Format (Strict JSON):
    Return **only the final JSON** after reasoning in this format:

    {{
    "nodes": [
        {{"type": "Initial Node", "name": "start"}},
        {{"type": "Action Node", "name": "selectTicketType"}},
        ...
        {{"type": "Final Node", "name": "end"}}
    ]
    }}

    ---

    **Identified Actions:**
    {actions}

    Now think step-by-step and define the most accurate set of activity nodes.
    """


    try:
        # Generate response using Gemini model
        model = genai.GenerativeModel("gemini-1.5-flash")
        response = model.generate_content(prompt, generation_config={"temperature": 0.0, "top_p": 1, "top_k": 1})
        
        # Debugging: Print raw response
        response_text = response.text.strip()
        print("Raw Response from Gemini:\n", response_text)

        # Clean up response (remove markdown formatting)
        cleaned_text = re.sub(r"```json|```", "", response_text).strip()

        # Extract the JSON part using regex
        json_match = re.search(r"\{\s*\"nodes\".*\}", cleaned_text, re.DOTALL)
        if json_match:
            json_str = json_match.group(0)

            try:
                output = json.loads(json_str)
                return output.get("nodes", [])
            except json.JSONDecodeError as e:
                print("JSON parsing error:", str(e))
                return {}
        else:
            print("No JSON found in response.")
            return {}

    except Exception as e:
        print("Error:", str(e))
        return {}

# Generate Activity Nodes using Gemini
identified_nodes = define_activity_nodes(identified_actions)

# Display Result
print("\nIdentified Activity Nodes:", identified_nodes)


Raw Response from Gemini:
 ```json
{
  "nodes": [
    {
      "type": "Initial Node",
      "name": "startPurchase"
    },
    {
      "type": "Action Node",
      "name": "selectTicketType"
    },
    {
      "type": "Action Node",
      "name": "selectTicketQuantity"
    },
    {
      "type": "Decision Node",
      "name": "choosePaymentMethod"
    },
    {
      "type": "Action Node",
      "name": "processCashPayment"
    },
    {
      "type": "Action Node",
      "name": "processCardPayment"
    },
    {
      "type": "Merge Node",
      "name": "paymentComplete"
    },
    {
      "type": "Action Node",
      "name": "printTicket"
    },
    {
      "type": "Action Node",
      "name": "sendTransactionConfirmation"
    },
    {
      "type": "Final Node",
      "name": "purchaseComplete"
    },
    {
      "type": "Decision Node",
      "name": "handlePaymentFailure"
    },
    {
      "type": "Action Node",
      "name": "handleInsufficientFunds"
    },
    {
      "type": "Ac

### Step 3 - Establish Control Flow

In [17]:
def establish_control_flow(nodes, actions):
    """
    Establishes control flow between activity nodes using Gemini API with CoT prompting.
    """
    prompt = f"""
    You are an expert software analyst. Your task is to define the **control flow transitions** between the nodes of an activity diagram.
    You will use the list of nodes and the corresponding actions to determine how the flow of the system progresses.
    Use the following chain-of-thought reasoning process to ensure correctness and completeness:

    ---

    ### 🧠 Step-by-Step Reasoning:

    1. **Start the Flow:**
    - Identify the **Initial Node**.
    - Connect it to the first logical **Action Node**.

    2. **Establish Sequential Flows:**
    - For each **Action Node**, determine what logically comes next.
    - Connect nodes with a `"type": "direct"` unless a control condition applies.

    3. **Branching with Decision Nodes:**
    - If a **Decision Node** exists:
        - Link it to each possible outcome using `"type": "decision"`.
        - Ensure those branches lead to meaningful nodes (e.g., actions or merge nodes).

    4. **Merging Paths:**
    - If different branches rejoin, use a **Merge Node**.
    - Link all decision outcomes back to the merge.

    5. **Parallelism (Optional):**
    - If the system performs tasks in parallel, use a **Fork Node**.
    - Use a **Join Node** to synchronize.

    6. **End the Flow:**
    - Identify the **Final Node**.
    - Ensure that all flows eventually connect to it.

    ---

    ### 🧾 Output Format (Strict JSON Only):
    Return **only the final JSON** in this format:

    {{
    "control_flow": [
        {{"source": "start", "target": "selectTicketType", "type": "direct"}},
        {{"source": "selectTicketType", "target": "choosePaymentMethod", "type": "direct"}},
        ...
        {{"source": "printTicket", "target": "end", "type": "direct"}}
    ]
    }}

    ---

    ### 🧩 Defined Nodes:
    {nodes}

    ### 🔨 Input Actions:
    {actions}

    Now, think step-by-step and establish a logically complete control flow using the rules above.
    """


    try:
        # Generate response using Gemini model
        model = genai.GenerativeModel("gemini-1.5-flash")
        response = model.generate_content(prompt, generation_config={"temperature": 0.7, "top_p": 1, "top_k": 1})
        
        # Debugging: Print raw response
        response_text = response.text.strip()
        print("Raw Response from Gemini:\n", response_text)

       # Clean up response (remove markdown formatting)
        cleaned_text = re.sub(r"```json|```", "", response_text).strip()

        # Extract the JSON part using regex
        json_match = re.search(r"\{\s*\"control_flow\".*\}", cleaned_text, re.DOTALL)
        if json_match:
            json_str = json_match.group(0)

            try:
                output = json.loads(json_str)
                return output.get("control_flow", [])
            except json.JSONDecodeError as e:
                print("JSON parsing error:", str(e))
                return {}
        else:
            print("No JSON found in response.")
            return {}

    except Exception as e:
        print("Error:", str(e))
        return {}

# Generate Control Flow using Gemini
control_flow = establish_control_flow(identified_nodes, identified_actions)

# Display Result
print("\nGenerated Control Flow:", control_flow)


Raw Response from Gemini:
 ```json
{
  "control_flow": [
    {"source": "startPurchase", "target": "selectTicketType", "type": "direct"},
    {"source": "selectTicketType", "target": "selectTicketQuantity", "type": "direct"},
    {"source": "selectTicketQuantity", "target": "choosePaymentMethod", "type": "direct"},
    {"source": "choosePaymentMethod", "target": "processCashPayment", "type": "decision"},
    {"source": "choosePaymentMethod", "target": "processCardPayment", "type": "decision"},
    {"source": "processCashPayment", "target": "paymentComplete", "type": "direct"},
    {"source": "processCardPayment", "target": "paymentComplete", "type": "direct"},
    {"source": "paymentComplete", "target": "printTicket", "type": "direct"},
    {"source": "printTicket", "target": "sendTransactionConfirmation", "type": "direct"},
    {"source": "sendTransactionConfirmation", "target": "purchaseComplete", "type": "direct"},
    {"source": "choosePaymentMethod", "target": "handlePaymentFailur

### Step 4 - Generate plantUML Script

In [ ]:
import os

def generate_plantuml(nodes, control_flow, output_file="activity_diagram.puml"):
    """
    Generates a PlantUML activity diagram script from identified nodes and control flow.
    """
    
    prompt = f"""
    You are an expert UML Diagram Developer. Your task is to generate plantUML script for defined activity nodes and control flows for an activity diagram.

    **Defined Nodes:**
    {nodes}

    **Defined Control_flow**
    {control_flow}

    Now generate the PlantUML activity diagram. 
    **Return only the PlantUML code.**  
    """
    

    try:
        # Generate response using Gemini model
        model = genai.GenerativeModel("gemini-1.5-flash")
        response = model.generate_content(prompt, generation_config={"temperature": 0.0, "top_p": 1, "top_k": 1})
        
        # Debugging: Print raw response
        response_text = response.text.strip()
        print("Raw Response from Gemini:\n", response_text)

        # Clean up response (remove markdown formatting)
        cleaned_text = re.sub(r"```plantuml|```", "", response_text).strip()

        # Additional Debugging: Print cleaned text before saving
        print("Cleaned Code:\n", repr(cleaned_text))

        # ✅ Save to file
        with open(output_file, "w") as file:
            file.write(cleaned_text)

        print(f"✅ PlantUML file '{output_file}' generated successfully.")

        return cleaned_text

    except Exception as e:
        print("Error:", str(e))
        return None

# Generate UML
plantuml_code = generate_plantuml(identified_nodes, control_flow)

# ✅ To render, use:
# `plantuml output.puml`

Raw Response from Gemini:
 ```plantuml
@startuml
start

:startPurchase;

:selectTicketType;

:selectTicketQuantity;

:choosePaymentMethod;

if (choosePaymentMethod) then (Cash)
  :processCashPayment;
  :paymentComplete;
elseif (choosePaymentMethod) then (Card)
  :processCardPayment;
  :paymentComplete;
else (Payment Failure)
  :handlePaymentFailure;
  if (handlePaymentFailure) then (Insufficient Funds)
    :handleInsufficientFunds;
    :purchaseComplete;
  elseif (handlePaymentFailure) then (Transaction Abort)
    :handleTransactionAbort;
    :purchaseComplete;
  endif
endif

:printTicket;

:sendTransactionConfirmation;

:purchaseComplete;

stop

@enduml
```
Cleaned Code:
 '@startuml\nstart\n\n:startPurchase;\n\n:selectTicketType;\n\n:selectTicketQuantity;\n\n:choosePaymentMethod;\n\nif (choosePaymentMethod) then (Cash)\n  :processCashPayment;\n  :paymentComplete;\nelseif (choosePaymentMethod) then (Card)\n  :processCardPayment;\n  :paymentComplete;\nelse (Payment Failure)\n  :handlePa

### Debugging...

In [9]:
import os

def generate_plantuml(nodes, control_flow, output_file="debug.puml"):
    """
    Generates a PlantUML activity diagram script from identified nodes and control flow.
    """
    
    prompt = f"""
    You are an expert UML Diagram Developer. Your task is to generate plantUML script for defined activity nodes and control flows for an activity diagram.

    **Defined Nodes:**
    {nodes}

    **Defined Control_flow**
    {control_flow}

    Now generate the PlantUML activity diagram. 
    **Return only the PlantUML code.**  
    """
    

    try:
        # Generate response using Gemini model
        model = genai.GenerativeModel("gemini-1.5-flash")
        response = model.generate_content(prompt, generation_config={"temperature": 0.7, "top_p": 1, "top_k": 1})
        
        # Debugging: Print raw response
        response_text = response.text.strip()
        print("Raw Response from Gemini:\n", response_text)

        # Clean up response (remove markdown formatting)
        cleaned_text = re.sub(r"```plantuml|```", "", response_text).strip()

        # Additional Debugging: Print cleaned text before saving
        print("Cleaned Code:\n", repr(cleaned_text))

        # ✅ Save to file
        with open(output_file, "w") as file:
            file.write(cleaned_text)

        print(f"✅ PlantUML file '{output_file}' generated successfully.")

        return cleaned_text

    except Exception as e:
        print("Error:", str(e))
        return None

# Generate UML
plantuml_code = generate_plantuml(identified_nodes, control_flow)

# ✅ To render, use:
# `plantuml output.puml`

Raw Response from Gemini:
 ```plantuml
@startuml
start

:startPurchase;

:selectTicketType;

:selectTicketQuantity;

:choosePaymentMethod;

if (choosePaymentMethod) then (Cash)
  :processCashPayment;
  :paymentComplete;
else if (choosePaymentMethod) then (Card)
  :processCardPayment;
  :paymentComplete;
else (Payment Failure)
  :handlePaymentFailure;
  if (handlePaymentFailure) then (Insufficient Funds)
    :handleInsufficientFunds;
    :purchaseComplete;
  else (Transaction Abort)
    :handleTransactionAbort;
    :purchaseComplete;
  endif
endif

:printTicket;

:sendTransactionConfirmation;

:purchaseComplete;

stop

@enduml
```
Cleaned Code:
 '@startuml\nstart\n\n:startPurchase;\n\n:selectTicketType;\n\n:selectTicketQuantity;\n\n:choosePaymentMethod;\n\nif (choosePaymentMethod) then (Cash)\n  :processCashPayment;\n  :paymentComplete;\nelse if (choosePaymentMethod) then (Card)\n  :processCardPayment;\n  :paymentComplete;\nelse (Payment Failure)\n  :handlePaymentFailure;\n  if (handleP